<a href="https://colab.research.google.com/github/alvaroiramirez/Harvard_DataMining_Business_Student/blob/master/Case_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#' Author: Ted Kwartler
#' Date: 2-14-2019
#' Purpose: OKCupid Case Supplemental
#'
#'
#' Student: Alvaro Ramirez
#' Class: CSCI E-96

#' Exploratory Data Analysis (EDA)
#' -------------------------------


# ENVIRONMENT SETUP

# libraries
library(dplyr)
library(ggplot2)
library(ggthemes)
library(leaflet)
library(leaflet.extras)
library(mapproj)
library(lubridate)
library(radiant.data)
library(DataExplorer)
library(stringr)
library(esquisse)
library(tidyr)
library(knitr)

# Set working directory
setwd("/Users/alvaroramirez/Library/CloudStorage/OneDrive-Personal/estudio/Harvard/Classes/CSCI E-96/CSCI E-96/Cases/Fall/I Ok Cupid")


# DATA IMPORT

# Load OK Cupid data into memory
profiles <- read.csv("profiles.csv", stringsAsFactors = FALSE)
latlon <- read.csv("LatLon.csv", stringsAsFactors = FALSE)


# DATA STRUCTURE

# profiles.csv
summary(profiles)

# LatLon.csv
summary(latlon)

# Explore profiles
str(profiles)

# Explore latlon
str(latlon)

# NOTE: latlon and profiles have a one-to-many relationship. The field used to
#       generate this link between both tables is 'location'.

# Summarize the number of missing values in each column
missing_summary <- sapply(profiles, function(x) sum(is.na(x)))
missing_percentage <- sapply(profiles, function(x) mean(is.na(x)) * 100)

# Create a summary table of missing values
missing_data <- data.frame(
  Column = names(profiles),
  MissingValues = missing_summary,
  MissingPercentage = missing_percentage
)

print(missing_data)


# Values used in categorical columns

# Function to calculate count and percentage for each unique value, handling NA values
create_df_with_counts <- function(column) {

  # Replace NA values with a placeholder ('<NA>')
  column_clean <- ifelse(is.na(column), '<NA>', column)

  # Get unique values including '<NA>' placeholder for NA
  unique_values <- unique(column_clean)

  # Create a dataframe with the unique values and their counts
  df <- data.frame(
    UniqueValue = unique_values,
    Count = sapply(unique_values, function(x) sum(column_clean == x))
  )

  # Calculate the percentage for each value
  df$Percentage <- (df$Count / length(column)) * 100

  # Return both the dataframe and the number of unique categories
  return(list(df = df, num_categories = length(unique_values)))
}

# Function to print the number of categories and the dataframe for each column
print_df_with_category_count <- function(df_list, column_name) {
  cat("Number of categories for '", column_name, "': ", df_list$num_categories, "\n", sep = "")
  print(df_list$df)
  cat("\n")
}

# Create dataframes for each specified field with counts and percentages, and print the results

# 'body_type'
df_body_type <- create_df_with_counts(profiles$body_type)
print_df_with_category_count(df_body_type, 'body_type')

# 'diet'
df_diet <- create_df_with_counts(profiles$diet)
print_df_with_category_count(df_diet, 'diet')

# 'drinks'
df_drinks <- create_df_with_counts(profiles$drinks)
print_df_with_category_count(df_drinks, 'drinks')

# 'drugs'
df_drugs <- create_df_with_counts(profiles$drugs)
print_df_with_category_count(df_drugs, 'drugs')

# 'education'
df_education <- create_df_with_counts(profiles$education)
print_df_with_category_count(df_education, 'education')

# 'ethnicity'
df_ethnicity <- create_df_with_counts(profiles$ethnicity)
print_df_with_category_count(df_ethnicity, 'ethnicity')

# 'job'
df_job <- create_df_with_counts(profiles$job)
print_df_with_category_count(df_job, 'job')

# 'location'
df_location <- create_df_with_counts(profiles$location)
print_df_with_category_count(df_location, 'location')

# 'offspring'
df_offspring <- create_df_with_counts(profiles$offspring)
print_df_with_category_count(df_offspring, 'offspring')

# 'orientation'
df_orientation <- create_df_with_counts(profiles$orientation)
print_df_with_category_count(df_orientation, 'orientation')

# 'pets'
df_pets <- create_df_with_counts(profiles$pets)
print_df_with_category_count(df_pets, 'pets')

# 'religion'
df_religion <- create_df_with_counts(profiles$religion)
print_df_with_category_count(df_religion, 'religion')

# 'sex'
df_sex <- create_df_with_counts(profiles$sex)
print_df_with_category_count(df_sex, 'sex')

# 'sign'
df_sign <- create_df_with_counts(profiles$sign)
print_df_with_category_count(df_sign, 'sign')

# 'smokes'
df_smokes <- create_df_with_counts(profiles$smokes)
print_df_with_category_count(df_smokes, 'smokes')

# 'speaks'
df_speaks <- create_df_with_counts(profiles$speaks)
print_df_with_category_count(df_speaks, 'speaks')

# Save the 'df_speaks' dataframe as a CSV file
write.csv(df_speaks$df, file = "speaks.csv", row.names = FALSE)

# 'status'
df_status <- create_df_with_counts(profiles$status)
print_df_with_category_count(df_status, 'status')


### Language analysis - Start

# Load necessary libraries
library(dplyr)
library(tidyr)
library(stringr)

# Read the profiles.csv file
# profiles <- read.csv("profiles.csv")

# Function to process the 'speaks' column
process_speaks <- function(speaks) {
  if (is.na(speaks) || speaks == "") {
    # return("english")
    return("")
  }
  # Split the speaks column by comma
  languages <- unlist(strsplit(speaks, ","))
  # Remove leading and trailing whitespace
  languages <- str_trim(languages)
  # Remove duplicates
  languages <- unique(languages)
  return(paste(languages, collapse = ", "))
}

# Apply the function to the 'speaks' column
profiles$speaks <- sapply(profiles$speaks, process_speaks)

# 1. Count how many languages each person speaks
profiles <- profiles %>%
  mutate(language_count = sapply(strsplit(speaks, ", "), length))

# 2. Use 'english' if there is no value (already handled in process_speaks function)

# 3. Count how many people speak each language
language_counts <- profiles %>%
  separate_rows(speaks, sep = ", ") %>%
  group_by(speaks) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# 4. Count how many people speak each language indicating their level
language_level_counts <- profiles %>%
  separate_rows(speaks, sep = ", ") %>%
  mutate(language = str_extract(speaks, "^[^()]+"),
         level = str_extract(speaks, "\\(([^)]+)\\)")) %>%
  group_by(language, level) %>%
  summarise(count = n()) %>%
  arrange(language, desc(count))

# Print the results
print(profiles)
print(language_counts)
print(language_level_counts)

# Exporting results
write.csv(language_counts, "language_counts.csv", row.names = FALSE)
write.csv(language_level_counts, "language_level_counts.csv", row.names = FALSE)


# COUNT HOW MANY PEOPLE SPEAK EACH LANGUAGE

# Function to process the 'speaks' column
process_speaks <- function(speaks) {
  if (is.na(speaks) || speaks == "") {
    return("english")
  }
  # Split the speaks column by comma
  languages <- unlist(strsplit(speaks, ","))
  # Remove leading and trailing whitespace and any extra spaces
  languages <- str_trim(languages)
  languages <- gsub("\\s+", "", languages)
  # Remove duplicates
  languages <- unique(languages)
  return(paste(languages, collapse = ", "))
}

# Apply the function to the 'speaks' column
profiles$speaks <- sapply(profiles$speaks, process_speaks)

# 1. Count how many languages each person speaks
profiles <- profiles %>%
  mutate(language_count = sapply(strsplit(speaks, ", "), length))

# 2. Use 'english' if there is no value (already handled in process_speaks function)

# 4. Count how many people speak each language indicating their level
language_level_counts <- profiles %>%
  separate_rows(speaks, sep = ", ") %>%
  mutate(language = str_extract(speaks, "^[^()]+"),
         level = str_extract(speaks, "\\(([^)]+)\\)")) %>%
  group_by(language, level) %>%
  summarise(count = n()) %>%
  arrange(language, desc(count))

# Print the totals for each language grouping all levels
totals_per_language <- language_level_counts %>%
  group_by(language) %>%
  summarise(total_count = sum(count)) %>%
  arrange(desc(total_count))

print(totals_per_language)

# Save the results in a CSV file
write.csv(totals_per_language, "totals_per_language.csv", row.names = FALSE)



### ### Language analysis - End

## -----------------


# Read the profiles.csv file
profiles <- read.csv("profiles.csv")

# Define the order of fluency levels
fluency_levels <- c(NA, "(poorly)", "(okay)", "(fluently)")


# Function to process the 'speaks' column
process_speaks <- function(speaks) {

  # print(paste("0 - speaks: ", speaks, sep = "")) ### STEP 0

  if (is.na(speaks) || speaks == "") {
    # return("english")
    return("")
  }

  # print(paste("1 - speaks: ", speaks, sep = "")) ### STEP 1

  # Split the speaks column by comma
  languages <- unlist(strsplit(speaks, ","))

  # print(paste("2 - languages: ", languages, sep = "")) ### STEP 2

  # Remove leading and trailing whitespace and any extra spaces
  languages <- str_trim(languages)
  languages <- gsub("\\s+", "", languages)

  # print(paste("3 - languages: ", languages, sep = "")) ### STEP 3

  # Create a data frame of languages and levels
  languages_df <- data.frame(language = str_extract(languages, "^[^()]+"),
                             level = str_extract(languages, "\\(([^)]+)\\)"))

  # print(paste("4 - languages_df: ", languages_df, sep = "")) ### STEP 4
  # kable(languages_df, caption = 'STEP 4')

  # Ensure 'english' is included with no level if not already present
  if (!any(grepl("^english", tolower(languages_df$language)))) {
    languages_df <- rbind(languages_df, data.frame(language = "english", level = NA))
  }

  # print(paste("5 - languages_df: ", languages_df, sep = "")) ### STEP 5
  # kable(languages_df, caption = 'STEP 5')


  # Remove duplicates and keep the highest level, handle NA levels correctly
  cleaned_languages_df <- languages_df %>%
    mutate(level = factor(level, levels = fluency_levels, ordered = TRUE)) %>%
    group_by(language) %>%
    filter(if(all(is.na(level))) TRUE else level == max(level, na.rm = TRUE)) %>%
    distinct(language, .keep_all = TRUE) %>%  # Remove duplicates after filtering
    ungroup()  # Ungroup the dataframe

  # Print the cleaned dataframe
  # print(paste("6 - languages_df: ", cleaned_languages_df, sep = "")) ### STEP 6
  # kable(cleaned_languages_df, caption = 'STEP 6')

  # Combine languages and levels back into a single string
  languages <- paste(cleaned_languages_df$language, ifelse(is.na(cleaned_languages_df$level), "", paste0(cleaned_languages_df$level)), sep = "", collapse = ", ")

  # print(paste("7 - languages: ", languages, sep = "")) ### STEP 7

  return(languages)
}


### TEST ###
str <- 'english, english (okay), spanish (okay), spanish (poorly), french, italian (fluently), chinese (fluently), chinese (fluently), thai, thai'
print(process_speaks(str))
############


# Create a new dataset to avoid overwriting the original 'profiles' dataset
profiles_processed <- profiles ########
profiles_processed$speaks <- sapply(profiles_processed$speaks, process_speaks)

# 1. Count how many languages each person speaks
profiles_processed <- profiles_processed %>%
  mutate(language_count = sapply(strsplit(speaks, ", "), length))

# 4. Count how many people speak each language indicating their level
language_level_counts <- profiles_processed %>%
  separate_rows(speaks, sep = ", ") %>%
  mutate(language = str_extract(speaks, "^[^()]+"),
         level = str_extract(speaks, "\\(([^)]+)\\)")) %>%
  group_by(language, level) %>%
  summarise(count = n_distinct(row_number())) %>%
  arrange(language, desc(count))

# Calculate the totals for each language spoken
totals_per_language <- language_level_counts %>%
  group_by(language) %>%
  summarise(total_count = sum(count)) %>%
  arrange(desc(total_count))

# Print the results
print(language_level_counts)
print(totals_per_language)

# Save the results in CSV files
write.csv(language_level_counts, "speaks_language_level_counts.csv", row.names = FALSE)
write.csv(totals_per_language, "speaks_totals_per_language.csv", row.names = FALSE)
write.csv(profiles_processed, "speaks_profiles_processed.csv", row.names = FALSE)




## -----------------


# Let's start with analyzing the 'age' column

# Summary of age column
summary(profiles$age)

# Check how many records have unrealistic values (age < 18 or age > 100)
invalid_age <- profiles %>%
  filter(age < 18 | age > 100)

# Calculate percentage of invalid age records
invalid_age_count <- nrow(invalid_age)
invalid_age_percentage <- (invalid_age_count / nrow(profiles)) * 100

cat("Number of invalid age records: ", invalid_age_count, "\n")
cat("Percentage of invalid age records: ", invalid_age_percentage, "%\n")

# Visualize age data with a boxplot to identify outliers
ggplot(profiles, aes(x = factor(0), y = age)) +
  geom_boxplot(fill = "lightblue") +
  theme_minimal() +
  labs(title = "Boxplot of Age", x = "", y = "Age")

# Plot age distribution with a histogram
ggplot(profiles, aes(x = age)) +
  geom_histogram(binwidth = 1, fill = "skyblue", color = "black") +
  theme_minimal() +
  labs(title = "Age Distribution", x = "Age", y = "Count")

# Analysis of height column

# Summary of height column
summary(profiles$height)

# Check for invalid height values (less than 50 inches or greater than 95 inches)
invalid_height <- profiles %>%
  filter(height < 50 | height > 95)

# Calculate percentage of invalid height records
invalid_height_count <- nrow(invalid_height)
invalid_height_percentage <- (invalid_height_count / nrow(profiles)) * 100

cat("Number of invalid height records: ", invalid_height_count, "\n")
cat("Percentage of invalid height records: ", invalid_height_percentage, "%\n")

# Visualize height data with a boxplot to detect outliers
ggplot(profiles, aes(x = factor(0), y = height)) +
  geom_boxplot(fill = "salmon") +
  theme_minimal() +
  labs(title = "Boxplot of Height", x = "", y = "Height (in inches)")

# Plot height distribution with a histogram
ggplot(profiles, aes(x = height)) +
  geom_histogram(binwidth = 1, fill = "salmon", color = "black") +
  theme_minimal() +
  labs(title = "Height Distribution", x = "Height", y = "Count")

# Income analysis

# Summary of income column
summary(profiles$income)

# Check for invalid income values (income > 1,000,000)
invalid_income <- profiles %>%
  filter(income > 1000000)

# Calculate percentage of invalid income records
invalid_income_count <- nrow(invalid_income)
invalid_income_percentage <- (invalid_income_count / nrow(profiles)) * 100

cat("Number of invalid income records: ", invalid_income_count, "\n")
cat("Percentage of invalid income records: ", invalid_income_percentage, "%\n")

# Visualize income with a boxplot
ggplot(profiles, aes(x = factor(0), y = income)) +
  geom_boxplot(fill = "lightgreen") +
  theme_minimal() +
  labs(title = "Boxplot of Income", x = "", y = "Income")

# Plot income distribution
ggplot(profiles, aes(x = income)) +
  geom_histogram(binwidth = 10000, fill = "lightgreen", color = "black") +
  theme_minimal() +
  labs(title = "Income Distribution", x = "Income", y = "Count")

# Now let's check how many records have complete data (i.e., no missing values in any column)
complete_records <- profiles[complete.cases(profiles), ]

# Calculate the percentage of records with complete data
complete_data_count <- nrow(complete_records)
complete_data_percentage <- (complete_data_count / nrow(profiles)) * 100

cat("Number of complete records: ", complete_data_count, "\n")
cat("Percentage of complete records: ", complete_data_percentage, "%\n")

# Now, we perform some combined column analysis. Let's analyze 'age' vs 'income'
ggplot(profiles, aes(x = age, y = income)) +
  geom_point(alpha = 0.5, color = "blue") +
  theme_minimal() +
  labs(title = "Age vs Income", x = "Age", y = "Income")

# Also analyze the relationship between 'education' and 'income'
ggplot(profiles, aes(x = education, y = income, fill = education)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Education Level vs Income", x = "Education Level", y = "Income")

# Now analyze 'diet' vs 'drinks' to see the distribution between these two habits
diet_drinks_table <- table(profiles$diet, profiles$drinks)
print(diet_drinks_table)

# Plot diet vs drinks
ggplot(profiles, aes(x = diet, fill = drinks)) +
  geom_bar(position = "dodge") +
  theme_minimal() +
  labs(title = "Diet vs Drinking Habits", x = "Diet", y = "Count")

# Check the geographic distribution by merging with the LatLon dataset
profiles_latlon <- merge(profiles, latlon, by = "location")

# Summarize combined dataset after merging
summary(profiles_latlon)

# Map visualization: Display users on a map based on location
leaflet(profiles_latlon) %>%
  addTiles() %>%
  addCircleMarkers(~lon, ~lat, radius = 2, color = "blue",
                   fillOpacity = 0.5, popup = ~location) %>%
  addProviderTiles(providers$Stamen.TonerLite)

# Analyzing 'sex' and 'income' combination to check income distribution by gender
ggplot(profiles, aes(x = sex, y = income, fill = sex)) +
  geom_boxplot() +
  theme_minimal() +
  labs(title = "Income Distribution by Gender", x = "Gender", y = "Income")

# Analyzing 'last_online' column: Check activity in the last year
profiles$last_online <- as.Date(profiles$last_online, format = "%Y-%m-%d")

# Filter active users (who were online in the last year)
active_users <- profiles %>%
  filter(last_online > Sys.Date() - years(1))

# Plot number of active users by location
ggplot(active_users, aes(x = location)) +
  geom_bar(fill = "orange") +
  coord_flip() +
  theme_minimal() +
  labs(title = "Active Users by Location", x = "Location", y = "Count")

# Exporting cleaned and merged data for future use
write.csv(profiles_latlon, "profiles_latlon_cleaned.csv", row.names = FALSE)

# Generate summary report using DataExplorer
create_report(profiles_latlon)

# End of R code

##################



##### I would do some basic EDA and plotting of individual vars then move to more complex interactions # nolint: line_length_linter.
table(profiles$orientation)
hist(profiles$age)

##### Example 2 way EDA
table(profiles$age, profiles$orientation)

#### Missing in income & quick mean imputation example; you can still use vtreat instead to clean all
#### this data but we are only exploring not modeling so maybe dont do it for this case. # nolint: line_length_linter.
sum(is.na(profiles$income))
profiles$income[is.na(profiles$income)] <- mean(profiles$income, na.rm = TRUE)

##### Feature Engineer relationship status & education if you thought there was a connection # nolint: line_length_linter.
profiles$statEDU <- paste(profiles$status, profiles$education, sep = "_")
table(profiles$statEDU)

##### Enrich with one of the new data sets, you may want to do this with the other csv # nolint: line_length_linter.
moreData <- left_join(profiles, latlon, by = "location") # nolint: object_name_linter, line_length_linter.
head(moreData)

#### You can use complete.cases() to identify records without NA if that is the route
#### you want to explore.  Of course you can use a function covered in class to visualize
#### the variables with the hightest % of NA so you could drop those instead of all rows
#### with an NA. # nolint: line_length_linter.
completeMoreData <- moreData[complete.cases(moreData), ] # nolint: object_name_linter, line_length_linter.
completeMoreData
nrow(completeMoreData)
dim(completeMoreData)
# End


Artifacts
1. R code
2. Word document
3. PowerPoint presentation
4. Presenter's notes
5. YouTube video script
